<a href="https://colab.research.google.com/github/MayorovKonstantin/Test/blob/main/PetProject_of_HousePrice_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://drive.google.com/file/d/10V00AzU0Ns0s13Ps2o3G33vlDFGOfeKD/view?usp=sharing

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data

In [ ]:
!gdown --id 10V00AzU0Ns0s13Ps2o3G33vlDFGOfeKD

In [ ]:
! unzip /content/house-prices-advanced-regression-techniques.zip

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
from scipy.stats import norm, skew
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))
%matplotlib inline

#EDA


In [ ]:
train = pd.read_csv('/content/train.csv')

In [ ]:
test = pd.read_csv('/content/test.csv')

In [ ]:
train_ID = train['Id']
test_ID = test['Id']

train.drop('Id', axis = 1, inplace = True)
test.drop('Id', axis = 1, inplace = True)

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]

y_train = train.SalePrice.values

all_data = pd.concat((train,test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True) #убираем целевой признак из общего датафрейма

In [ ]:
all_data.columns

Обратимся к файлу data_discription чтобы понять значение и выделить наиболее важные признаки (предоставлен на Kaggle)



1. Выделяем наиболее влиятельные по нашему мнению:
OverallQual - Общее качество
YearBuilt - год постройки
TotalBsmSF - площадь подземной части
GrLivArea - площадь надземной части

2. Попробуем проанализировать зависимость цены дама от этих признаков

##Анализ цены



In [ ]:
sns.distplot(y_train)

Для более точной оценки распределения существует две характеристики

Skewness - коэффициент асимметрии (Насколько пик распределения отклоняется от центра)

Kurtosis - коэффициент эксцесса (Острота пика распределения)

In [ ]:
print('Skewness: {}'.format(train['SalePrice'].skew()))
print('Kurtosis: {}'.format(train['SalePrice'].kurt()))

##Определим зависимость цены от других признаков

Теперь когда мы разобрадись с анализом одной величины, попробуем посмотреть как она взаимодействует с другими величинами

Ранее мы выделили 4 признака, которые кажутся потенциально значимыми.

Два из них числовые(GrLivArea и TotalBsmtSF), другие два категориальные (OverallQual и YearBuilt).

##Числовые признаки

Начнем с числовых признаков. Хорошим способом посмотреть на зависимость двух признаков друг от друга является точечная диаграмма.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 5))

ax[0].scatter(train['GrLivArea'], train['SalePrice'])
ax[0].set_xlabel('GrLivArea')
ax[0].set_ylabel('SalePrice')

ax[1].scatter(train['TotalBsmtSF'], train['SalePrice'])
ax[1].set_xlabel('TotalBsmtSF')
ax[1].set_ylabel('SalePrice')

Из диаграмм становится видно, что цена действительно сильно зависит от этих двух признаков

Линейно в случае с GrLivArea и линейно или возможно экспоненциально в случае TotalBsmtSF 

##Категориальные признаки

Точечные диаграммы для категориальных признаков менее информативны, разуменее для их анализа будет использовать ящик с усами

Строим для каждой категории качества(1-10)

In [ ]:
data = pd.concat([train['SalePrice'], train['OverallQual']], axis=1)
f, ax = plt.subplots(figsize=(10, 7))
fig = sns.boxplot(x='OverallQual', y = 'SalePrice', data=data)

На графике очень хорошо видан экспоненциальная зависимость как по медиана, так и по границам квантилей. На основе этого можно предположить, что качество потенциально значимый фактор.

Аналогично проанализируем год постройки дома

In [ ]:
data = pd.concat([train['SalePrice'], train['YearBuilt']], axis=1)
f, ax = plt.subplots(figsize=(30, 8))
fig = sns.boxplot(x='YearBuilt', y = 'SalePrice', data=data)

C годом постройки не видно хорошо описываемой зависимости, однако новые дома все же имеют тенденцию стоить дороже старых.

##Общий анализ (систематический)

Теперь давайте отступим от анализа признаков, которые показались нам значимыми, и попробуем более общий подход.

##Матрица корреляции

Хорошим инструментом для анализа зависимостей между всеми признаками является матрица корреляции

In [ ]:
corrmat = train.corr()
f, ax = plt.subplots(figsize =(12,9))
sns.heatmap(corrmat, vmax = .8, square =True)

Пара высококорелирующих признаков может быть заменена одним.

Посмотрев на матрицу корреляции мы можем выделить наиболее значимые для предсказания признаки. Вместе с тем, мы можем найти пары признаков имеющих высокую корреляцию и исключить один из них их анализа. Что позволит выбрать наибольший набор главных признаков для дальнейшего анализа.


Посмотрит на первые 10 признаков, имеющих наибольшую корреляцию с ценой дома

In [ ]:
corrmat.sort_values(by=['SalePrice'], ascending=False)[['SalePrice']][:10]

Признаки OverallQual и GrLivArea нам знакомы и мы убедились в из значимости

Признаки GarageCars и GarageArea очень близки по смыслу и меют высокую корреляцию между друг другом , поэтому мы можем оставить только один из них для дальнейшего поиска зависимостей.

аналогичная ситуация обстоит с парами признаков GrLivArea и TotRmsAbvGrd, TotalBsmtSF и 1stFLrSF

В итоге для дальнейшего анализа мы можем остаивть только 6 главных признаков.

##Парные зависимости между признаками

Теперь, когда множество признаков сильно уменьшилось, мы можем посмотреть на их зависимости между друг другом

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(train[cols], size = 2.5)
plt.show()

На этом этапе можно совершить множество значимых наблюдений, но мы не будем затягивать анализ и сделаем только два.
-зависимости между признаками похожи на линейные или мягкие экспоненциальные.
- среди зависимостей можно увидеть множество "конусов", такие зависимости плохо описываются линейными, поэтому от них стоит избавиться с помощью специальных преобразований (Об этом позже)

##Пропуски
Попробуем избавиться от пропусков более осозданным методом, чем выбрасываение всех строк с пропусками. 

Начать стоит с того, чтобы оценить, какой процент пропусков содержится в данных по каждому признаку.

In [ ]:
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(30)

In [ ]:
total1 = all_data.isnull().sum().sort_values(ascending=False)
percent = (all_data.isnull().sum()/all_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total1, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(40)

Разумно будет отсечь часть признаков, имеющих очень высокий процент пропусков и не пытаться их заполнить. Выкинем все признаки имеющие процент пропуска больше 0.15

In [ ]:
train = train.drop((missing_data[missing_data['Percent']>0.15]).index,1)

In [ ]:
all_data = all_data.drop((missing_data[missing_data['Percent']>0.15]).index,1)

Остальные пропуски заполним соответствующими логике признака значениями 'пропуска'(0, 'None', etc).

In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
  train[col] = train[col].fillna('None')
  all_data[col] = all_data[col].fillna('None')

In [ ]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
  train[col] = train[col].fillna(0)
  all_data[col] = all_data[col].fillna(0)

In [ ]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
  train[col] = train[col].fillna(0)
  all_data[col] = all_data[col].fillna(0)

In [ ]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
  train[col] = train[col].fillna('None')
  all_data[col] = all_data[col].fillna('None')

In [ ]:
train['MasVnrType'] = train['MasVnrType'].fillna('None')
all_data['MasVnrType'] = all_data['MasVnrType'].fillna('None')

train['MasVnrArea'] = train['MasVnrArea'].fillna(0)
all_data['MasVnrArea'] = all_data['MasVnrArea'].fillna(0)

In [ ]:
train['Electrical'] = train['Electrical'].fillna(train ['Electrical'].mode()[0]) #заполняем модой
all_data['Electrical'] = all_data['Electrical'].fillna(all_data ['Electrical'].mode()[0]) #заполняем модой
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data ['MSZoning'].mode()[0]) #заполняем модой
all_data['Utilities'] = all_data['Utilities'].fillna(all_data ['Utilities'].mode()[0]) #заполняем модой
all_data['Functional'] = all_data['Functional'].fillna(all_data ['Functional'].mode()[0]) #заполняем модой
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data ['KitchenQual'].mode()[0]) #заполняем модой
all_data['SaleType'] = all_data['SaleType'].fillna(all_data ['SaleType'].mode()[0]) #заполняем модой
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data ['Exterior2nd'].mode()[0]) #заполняем модой
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data ['Exterior1st'].mode()[0]) #заполняем модой


##Нормальное распределение

Многие методы машинного обучения опираются на нормальное распределение. И в общем случае, желательно работать с данными распределение которых_нормально.

Посмотрим как распределены наши признаки

In [ ]:
sns.distplot(train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)

хорошо заметно, что распределение далеко от нормального и имеет высокий положительный коэффициент ассиметрии. В таком случае к распределению можно применить функцию логарифма и 'выпрямить" распределение.

In [ ]:
y_train = np.log(train['SalePrice'])

In [ ]:
sns.distplot(y_train, fit=norm)
fig = plt.figure()
res = stats.probplot(y_train, plot=plt)

На точечных диаграммах такие зависимости похоже на конусы, о которых упоминалось ранее.

Однако нам удается избавиться от них.

Применим этот подод к другим признакам имеющим положительный коэффициент ассиметрии

In [ ]:
train['GrLivArea'] = np.log(train['GrLivArea'])
all_data['GrLivArea'] = np.log(all_data['GrLivArea'])
train.loc[train['TotalBsmtSF'] > 0, 'TotalBsmtSF']=np.log(train['TotalBsmtSF'])#Пропустим случаи с 0 значениями
all_data.loc[all_data['TotalBsmtSF'] > 0, 'TotalBsmtSF']=np.log(all_data['TotalBsmtSF'])#Пропустим случаи с 0 значениями


Если теперь посмотреть на точечные диаграммы зависимостей, то можно увидеть, что конусов больше нет.

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(train[cols], size = 2.5)
plt.show()

##Категориальные признаки (кодирование)

Среди признаков есть те, что представлены числами, но на самом деле являются категориальными. Просто преобразуем их в строки.

In [ ]:
train['MSSubClass']=train['MSSubClass'].apply(str)
train['OverallCond']=train['OverallCond'].astype(str)
train['YrSold']=train['YrSold'].astype(str)
train['MoSold']=train['MoSold'].astype(str)
all_data['MSSubClass']=all_data['MSSubClass'].apply(str)
all_data['OverallCond']=all_data['OverallCond'].astype(str)
all_data['YrSold']=all_data['YrSold'].astype(str)
all_data['MoSold']=all_data['MoSold'].astype(str)

Также есть категориальные признаки, которые принципиальнобудут иметь больше смысла , если закодировать их с помощью метода label encoding

В случае с one-hot encoding теряется упорядоченность категорий, label encoding эту упорядоченность сохраняет

In [ ]:
cols = ( 'ExterQual', 'ExterCond', 'HeatingQC',  'KitchenQual',
        'Functional', 'LandSlope', 'LotShape',
       'PavedDrive', 'CentralAir', 'MSSubClass', 'OverallCond', 'MoSold')

for c in cols:
   lbl1 = LabelEncoder()
   lbl1.fit(list(all_data[c].values))
   all_data[c] = lbl1.transform(list(all_data[c].values))

Оставшиеся категориальные признаки закодируем методом one-hot encoder

In [ ]:
all_data = pd.get_dummies(all_data)

In [ ]:
train = all_data[:ntrain]
test = all_data[ntrain:]

#Модель

In [ ]:
import scipy.special

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso, BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

##Стекинг

На сегодняшний день существует множество методов решения регрессионных задач. Однако вместо того, чтобы выбирать один метод, мы выберем 4 сразу.

Не редко в машинном обучении используются ансамбли моделей, несколько моделей, предсказание которых усредняется.

Методов объединения моделей в ансамбли несколько, но в этот раз мы используем один - Стекинг.

За стекингом стоит очень простая идея. Каждая модель в ансамбле обучается отдельно, а после создается мета-модель, которая только на основе предсказаний моделей ансамбля предсказывает единое усредненное предсказание.

##Кросс-валидация

Перед переходом к стекингу и ансамблям изменим наш способ оценки работы модели

Обучать модель полностью на тренировочной выборке и тестировать ее на тренировочной выборке - это плохой подход. Если модель переобучится, то мы это никак не заметим.

Разделим тренировочнуб выборку данных на тренировочную и валидационную - более хороший подход.

Но наиболее хорошим вариантом будет использовать кросс-валидацию.

1. Данные делятся на несколько равных частей.
2. Первая часть выбирется валидационной.
3. Модель обучается на всех данных, кроме валидационной части.
4. Первые три шага повторяются, но на втором берется не первая часть, а вторая , потом третья и.т.д
5. Результаты оценки усредняются

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

n_folds = 5

def rmse_cv(model, X, y):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X.values)
    rmse = np.sqrt(-cross_val_score(model, X.values, y.values, scoring='neg_mean_squared_error', cv=kf))
    return rmse

##Регрессионные модели

Объявим четыре модели и оценим их точность

Отдельно стоит изучить выбор модели и подбор гипер-параметров

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.0004, random_state=1))

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0004, l1_ratio=.9, random_state=3))

In [ ]:
KRR = KernelRidge(alpha=0.5, kernel='polynomial', gamma=0.2, degree=1, coef0=3)

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state=5)

Модели lasso и ENet чувствительны в выбросам значений, поэтому данные сначала нормализуются с помощью RobustScaler

Оценим их точность по отдельности

In [ ]:
train_reset = train.reset_index(drop=True)
target_reset = y_train.reset_index(drop=True)

In [ ]:
score = rmse_cv(lasso,train,y_train).mean()
print('Lasso score: {}'.format(round(score, 5)))
score = rmse_cv(ENet,train, y_train).mean()
print('Enet score: {}'.format(round(score, 5)))
score = rmse_cv(KRR,train,y_train).mean()
print('KRR score: {}'.format(round(score, 5)))
score = rmse_cv(GBoost,train, y_train).mean()
print('Gboost score: {}'.format(round(score, 5)))



Видим что впереди находится GBoost

##Ансамбль(усреднение)

Теперь объединяем модели в ансамбли

Перед тем, как перейти к использованию мета-модели, попробуем просто усреднить их предсказания.

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models

    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]

        # Обучаем модель
        for model in self.models_:
            model.fit(X, y)

        return self

    def predict(self, X):
        # Находим предсказание для каждой модели
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])

        # Усредняем предсказание
        return np.mean(predictions, axis=1)

Посмотрим какую точность имеет такой ансамбль

In [ ]:
averaged_models = AveragingModels(models = (ENet, lasso, KRR, GBoost))
# сбрасываем индексы для train и target
train_reset = train.reset_index(drop=True)
target_reset = y_train.reset_index(drop=True)

# запускаем кросс-валидацию с новыми индексами
score = rmse_cv(averaged_models, train_reset, target_reset).mean()
print('Stacking score: {}'.format(round(score,5)))

In [ ]:
averaged_models.fit(train.values, y_train)

In [ ]:
averaged_pred = averaged_models.predict(test)

In [ ]:
salePrice_pred = np.exp(averaged_pred)

###Сабмит №1

In [ ]:
import os

# creating submission directory if it does not exist
if not os.path.exists('submission'):
    os.makedirs('submission')

sub = pd.DataFrame()
sub['Id']=test_ID
sub['SalePrice'] = salePrice_pred
sub.to_csv('submission/basic_submission4.csv', index=False)

При простом усреднении нам удалось добиться большей точности, чем лучшая модель давала по отдельности

Заменим усреднение на мета-модель

##Ансамбль с метамоделью

---



In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
  def __init__(self, base_models, meta_model, n_folds=5):
    self.base_models = base_models
    self.meta_model = meta_model
    self.n_folds = n_folds
    self.base_models_ = [list() for _ in self.base_models]

  def fit(self, X, y):
      self.base_models_ = [list() for x in self.base_models]
      self.meta_model_ = clone(self.meta_model)
      kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
      out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
      for i, model in enumerate(self.base_models):
        for train_index, holdout_index in kfold.split(X, y):
          instance = clone(model)
          self.base_models_[i].append(instance)
          instance.fit(X[train_index], y[train_index])
          y_pred = instance.predict(X[holdout_index])
          out_of_fold_predictions[holdout_index, i] = y_pred

      self.meta_model_.fit(out_of_fold_predictions, y)
      return self

  def predict(self, X):
    meta_features = np.column_stack([
        np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
        for base_models in self.base_models_])
    return self.meta_model_.predict(meta_features)

Посмотрим на точность такого ансамбля (в качестве мета-модели используется Lasso)

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models=(ENet, GBoost, KRR,lasso),
                                                 meta_model=lasso)
# сбрасываем индексы для train и target
train_reset = train.reset_index(drop=True)
target_reset = y_train.reset_index(drop=True)

# запускаем кросс-валидацию с новыми индексами
score = rmse_cv(stacked_averaged_models, train_reset, target_reset).mean()
print('Stacking score: {}'.format(round(score,5)))


In [ ]:
stacked_averaged_models.fit(train.values, y_train)

In [ ]:
stacked_pred = stacked_averaged_models.predict(test)

In [ ]:
salePrice_pred = np.exp(stacked_pred)

### Сабмит№2

In [ ]:
import os

# creating submission directory if it does not exist
if not os.path.exists('submission'):
    os.makedirs('submission')

sub = pd.DataFrame()
sub['Id']=test_ID
sub['SalePrice'] = salePrice_pred
sub.to_csv('submission/basic_submission_8.csv', index=False)